# DQN - Keras

- **설명** : https://keon.io/deep-q-learning/
- **코드** : https://github.com/keon/deep-q-learning/blob/master/dqn.py

In [1]:
import keras 

keras.__version__

Using TensorFlow backend.


'2.2.4'

In [2]:
import random
import gym
import numpy as np
from collections import deque
from keras import layers, models
from keras.optimizers import Adam

# 맷플롯립 설정
from IPython.display import HTML
%matplotlib inline
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
matplotlib.rc('font', family='AppleGothic')  # MacOS
# matplotlib.rc('font', family='Malgun Gothic')  # Windows
# matplotlib.rc('font', family='NanumBarunGothic') # Linux
plt.rcParams['axes.unicode_minus'] = False

## 1. Create DQNAgent Class

In [3]:
class DQNAgent:
    # Deep Q-learning Agent
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01 
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
        
    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = models.Sequential()
        model.add(layers.Dense(24, activation='relu', 
                               input_shape=(self.state_size,)))
        model.add(layers.Dense(24, activation='relu'))
        model.add(layers.Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            # random하게 행동
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action
    
    def replay(self, batch_size):
        # sample minibatch from the memory
        minibatch = random.sample(self.memory, batch_size)
        # 3xtract info from each memory
        for state, action, reward, next_state, done in minibatch:
            # if done, make our target reward
            target = reward
            if not done:
                # predict the future discounted reward
                target = (reward + self.gamma * 
                          np.amax(self.model.predict(next_state)[0]))
            
            # make the agent to approximately map
            # the current state to future discounted reward
            # We'll call that target_f
            target_f = self.model.predict(state)
            target_f[0][action] = target
            # Train the NN with the state and target_f
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def load(self, name):
        self.model.load_weights(name)
        
    def save(self, name):
        self.model.save_weights(name)

## 2. Training DQN

### 1) Jupyter Notebook Rendering & Animation

In [4]:
from render_notebook import render_cart_pole

def plot_cart_pole(env, obs):
    img = render_cart_pole(env, obs)
    plt.imshow(img)
    plt.axis("off")
    plt.show()
    
def update_scene(num, frames, patch):
    plt.close()  # 이전 그래프를 닫지 않으면 두 개의 그래프가 출력되는 matplotlib의 버그로 보입니다.
    patch.set_data(frames[num])
    return patch,

def plot_animation(frames, figsize=(5,6), repeat=False, interval=40):
    fig = plt.figure(figsize=figsize)
    patch = plt.imshow(frames[0])
    plt.axis('off')
    return animation.FuncAnimation(fig, update_scene, fargs=(frames, patch), 
                                   frames=len(frames), repeat=repeat, interval=interval)

### 2) Training

In [5]:
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
print('state_size :', state_size)

state_size : 4


In [6]:
action_size = env.action_space.n
print('action_size :', action_size)

action_size : 2


In [7]:
agent = DQNAgent(state_size, action_size)

In [8]:
done = False
batch_size = 32
EPISODES = 100
frames = []

for e in range(EPISODES):
    state = env.reset()
    obs = state
    state = state.reshape([1, state_size])  # [1, 4]
    for time in range(500):
        img = render_cart_pole(env, obs)  # plotting
        frames.append(img)
        
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10  # penalty
        next_state = next_state.reshape([1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print('episode: {}/{}, score: {}, e: {:.2}'
                  .format(e, EPISODES, time, agent.epsilon))
            break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
            
env.close()

episode: 0/100, score: 15, e: 1.0
episode: 1/100, score: 55, e: 0.82
episode: 2/100, score: 18, e: 0.75
episode: 3/100, score: 26, e: 0.66
episode: 4/100, score: 63, e: 0.48
episode: 5/100, score: 10, e: 0.46
episode: 6/100, score: 15, e: 0.42
episode: 7/100, score: 12, e: 0.4
episode: 8/100, score: 11, e: 0.38
episode: 9/100, score: 10, e: 0.36
episode: 10/100, score: 10, e: 0.34
episode: 11/100, score: 9, e: 0.33
episode: 12/100, score: 28, e: 0.28
episode: 13/100, score: 9, e: 0.27
episode: 14/100, score: 10, e: 0.26
episode: 15/100, score: 9, e: 0.25
episode: 16/100, score: 11, e: 0.23
episode: 17/100, score: 9, e: 0.22
episode: 18/100, score: 10, e: 0.21
episode: 19/100, score: 15, e: 0.2
episode: 20/100, score: 10, e: 0.19
episode: 21/100, score: 22, e: 0.17
episode: 22/100, score: 22, e: 0.15
episode: 23/100, score: 10, e: 0.14
episode: 24/100, score: 10, e: 0.14
episode: 25/100, score: 9, e: 0.13
episode: 26/100, score: 28, e: 0.11
episode: 27/100, score: 16, e: 0.1
episode: 28

In [2]:
# video = plot_animation(frames, figsize=(6,4))
# HTML(video.to_html5_video())  # HTML5 동영상으로 만들어 줍니다